In [1]:
# default_exp viseme_image.model

# Image model

> Conv net, trained with fastai running in onnx.

In [2]:
#export
from expoco.viseme_image.data import *
from pathlib import Path
import numpy as np
import cv2, onnxruntime

# Prepare data for inference

We need to replicate what fastai data loaders do ...

In [3]:
#export
def prepare_for_inference(image, image_size=256):
    "Convert a cv2 style image to something that can be used as input to a CNN"
    if image.shape[0] > image_size:
        image = ImageHelper().face_crop(image)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = np.transpose(image, (2, 0, 1))
    image = (image/255.)
    image = image.astype(np.float32)
    return image

In [4]:
_image = np.zeros((4, 5, 3), np.uint8)
B, G, R = 255, 125, 0
_image[:,] = B, G, R
_image = prepare_for_inference(_image)
assert _image.shape == (3, 4, 5)
assert np.allclose(_image[0], R/255)
assert np.allclose(_image[1], G/255)
assert np.allclose(_image[2], B/255)

In [9]:
#export
class VisemeClassifier:
    def __init__(self, model_path, image_size=256):
        self.model = onnxruntime.InferenceSession(str(model_path))
        self.image_size = image_size
        self.image_helper = ImageHelper()
        self.vocab = ['AH', 'EE', 'NO_EXPRESSION', 'OO']
        self.item_queue = []
    def _to_image(self, item):
        return self.image_helper.read_image(item) if isinstance(item, (str, Path)) else item
    def predict(self, items=None):
        if items is None:
            items = self.item_queue
            self.item_queue = []
        else:
            items = [self._to_image(i) for i in items]
            items = [prepare_for_inference(i, self.image_size) for i in items]
        items = np.array(items)
        model_output = self.model.run(None, {'input': items})
        output = model_output[0]
        class_ids = np.argmax(output, axis=1)
        class_names = [self.vocab[class_id] for class_id in class_ids]
        return class_names
    def queue_item(self, item):
        self.item_queue.append(prepare_for_inference(self._to_image(item), self.image_size))

`queue_item` prepares for inference as this prep is quite often slower that actually running inference

In [10]:
model_path = Path('models/model_20211202_143854/resnet_3_256_256.onnx')
img_path = Path('test/data/raw_images')
imgs = [img_path/'oo_1.png', img_path/'oo_2.png', img_path/'ee_1.png', img_path/'ee_2.png']
img_path = Path('test/data/processed_images')
imgs += [img_path/'ah_1.png', img_path/'ah_2.png']
viseme_classifier = VisemeClassifier(model_path)
class_names = viseme_classifier.predict(imgs)
for img in imgs:
    viseme_classifier.queue_item(img)
class_names2 = viseme_classifier.predict()
assert class_names == class_names2 == ['OO', 'OO', 'EE', 'EE', 'AH', 'AH']

In [11]:
#hide
from nbdev.export import notebook2script
notebook2script()

Converted 00_core.ipynb.
Converted 01a_camera_capture.ipynb.
Converted 10a_viseme_tabular_identify_landmarks.ipynb.
Converted 10b_viseme_tabular_data.ipynb.
Converted 10d_viseme_tabular_model.ipynb.
Converted 10e_viseme_tabular_train_model.ipynb.
Converted 10f_viseme_tabular_test_model.ipynb.
Converted 11b_viseme_image_data.ipynb.
Converted 11d_viseme_image_model.ipynb.
Converted 11e_viseme_image_train_model.ipynb.
Converted 11f_viseme_image_test_model.ipynb.
Converted 20a_gui_capture_command.ipynb.
Converted 20a_gui_main.ipynb.
Converted 70_cli.ipynb.
Converted index.ipynb.
Converted project_lifecycle.ipynb.
